# 3.3 Search

Search is at the core of the RAG pattern. Precise, efficient, and consistent search is critical when implementing a solution based on RAG.

There are four types of search

## [The Role of Search](https://github.com/microsoft/rag-openai/blob/main/topics/RAG_EnablingSearch.md#the-role-of-search)

The main purpose of the search tool is to bring the first cut of relevant documents for further analysis by the large language model - it is there to filter the noise and reduce the result set for the model to summarize.

Search is at the heart of a RAG solution - it is the mechanism that ensures that the context that is sent to the prompt contains relevant information for it to answer the question.

## [Evaluating the Retrieval Component](https://github.com/microsoft/rag-openai/blob/main/topics/RAG_EnablingEvaluation.md#evaluating-the-retrieval-component)

Regarding the Retrieval component, the dataset is composed of question and citation instead of question and answer.

- question: the user question
- citation: the piece(s) of text that contains the relevant content to answer the user question
- answer: the final answer in a human readable/friendly format
  E
  valuating the Retrieval component means to evaluate if for a given query (user question) the search engine is returning the relevant citation(s).

Retrieval information is a well-known problem and the classic metrics are: Precision, Recall, F1 Score, Mean Average Precision (MAP), Mean Normalized Discounted Cumulative Gain (Mean NDCG) and Mean Reciprocal Rank (MRR). More details can be found at Evaluating Information Retrieval Models: A Comprehensive Guide to Performance Metrics.

[Evaluation Metrics](https://github.com/microsoft/rag-openai/blob/main/topics/RAG_EnablingEvaluation.md#evaluation-metrics)
Linnk: https://medium.com/@prateekgaurav/evaluating-information-retrieval-models-a-comprehensive-guide-to-performance-metrics-78aadacb73b4#:~:text=Evaluating%20Information%20Retrieval%20Models%3A%20A%20Comprehensive%20Guide%20to,...%206%206.%20Mean%20Reciprocal%20Rank%20%28MRR%29%20


### Perform a hybrid search

Hybrid Retrieval brings out the best of Keyword and Vector Search

Keyword and vector retrieval tackle search from different perspectives, which yield complementary capabilities. Vector retrieval semantically matches queries to passages with similar meanings. This is powerful because embeddings are less sensitive to misspellings, synonyms, and phrasing differences and can even work in cross lingual scenarios. Keyword search is useful because it prioritizes matching specific, important words that might be diluted in an embedding.

User search can take many forms. Hybrid retrieval consistently brings out the best from both retrieval methods across query types. With the most effective L1, the L2 ranking step can significantly improve the quality of results in the top positions.


In [1]:
# %run 2.rag-implementation-2.ipynb

In [2]:
# Pure Vector Search
query = "scalable storage solution"
query_embeddings = model.encode(query, normalize_embeddings=True)
vector_query = VectorizedQuery(
    vector=query_embeddings.tolist(), k_nearest_neighbors=3, fields="contentVector"
)

results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    top=3,
)

print_results(results)

NameError: name 'model' is not defined

### Perform a hybrid search

Hybrid Retrieval brings out the best of Keyword and Vector Search

Keyword and vector retrieval tackle search from different perspectives, which yield complementary capabilities. Vector retrieval semantically matches queries to passages with similar meanings. This is powerful because embeddings are less sensitive to misspellings, synonyms, and phrasing differences and can even work in cross lingual scenarios. Keyword search is useful because it prioritizes matching specific, important words that might be diluted in an embedding.

User search can take many forms. Hybrid retrieval consistently brings out the best from both retrieval methods across query types. With the most effective L1, the L2 ranking step can significantly improve the quality of results in the top positions.


### Perform a semantic hybrid search - Required Semantic Ranker enabled


In [4]:
%%capture --no-display

query = "what is azure sarch?"

query_embeddings = model.encode(query, normalize_embeddings=True)
vector_query = VectorizedQuery(
    vector=query_embeddings.tolist(), k_nearest_neighbors=3, fields="contentVector"
)

results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="my-semantic-config",
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3,
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

NameError: name 'model' is not defined